In [55]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas_ta as ta

# On lit et on met en forme le dataframe
# Séparation des colonnes csv, renommage des colonnes, suppression de la première ligne comme header
df = pd.read_csv("data/nzd_4h.csv",  sep="\t", names=['hour', 'open', 'high', 'low', 'close', 'tick', 'volume', 'spread'], header=0)

# On type la colonne date en datetime
df['datetime'] = pd.to_datetime(df.index + ' ' + df['hour'])
df['datetime'] = pd.to_datetime(df['datetime'],  format='%Y.%m.%d %H:%M')

# On set l'index à la colonne date
df.set_index('datetime', inplace=True)

# On garde uniquement la colonne close pour la stratégie
df = df[[ 'close']]

# Calcul du RSI
df['rsi'] = ta.rsi(df['close'], length=14)

df['signal'] = 0

# Signal d'achat
cross_up = (df['rsi'].shift(1) < 30) & (df['rsi'] > 30)
df.loc[cross_up, 'signal'] = 1
# Signal de vente
cross_down = (df['rsi'].shift(1) > 70) & (df['rsi'] < 70)
df.loc[cross_down, 'signal'] = -1

# On récupère le % de variation entre chaque trade
df['returns'] = df['close'].pct_change()

trades = []
in_trade = False
entry_index = None

for i in range(len(df)):
    row = df.iloc[i]
    
    if row['signal'] == 1 and not in_trade:
        entry_index = i
        in_trade = True

    elif row['signal'] == -1 and in_trade:
        exit_index = i
        in_trade = False

        # Extraire les retours entre entrée et sortie
        returns_slice = df['returns'].iloc[entry_index + 1 : exit_index + 1]
        trade_return = (1 + returns_slice).prod() - 1

        trades.append({
            'entry_date': df.index[entry_index],
            'exit_date': df.index[exit_index],
            'entry_price': df['close'].iloc[entry_index],
            'exit_price': df['close'].iloc[exit_index],
            'return': trade_return
        })
        

# On supprimes les lignes qui posssèdent des valeurs nulles
df = df.dropna()

trades_df = pd.DataFrame(trades)

total_trades = len(trades_df)
gagnants = trades_df[trades_df['return'] > 0].shape[0]
winrate = (gagnants / total_trades) * 100
total_gain = (1 + trades_df['return']).prod() - 1

print(f"Trades totaux : {sum_trades}")
print(f"Trades gagnants : {sum_win_trades}")
print(f"Gain total : {winrate:.2f}")
print(f"Pourcentage de gain : {total_gain:.2f}%")



Trades totaux : 28
Trades gagnants : 28
Gain total : 75.00
Pourcentage de gain : 0.07%
